In [123]:
# Import một số thư viện cần thiết cho việc thu thập
import json
import re
import pandas as pd
import scrapy

In [124]:
data=pd.read_json("cars_info.json")
df=pd.DataFrame(data)


In [125]:
df.to_csv('car.csv')


In [126]:
df.head(10)

,ID,Name,Price,Brand,Model,Variant,Series,Year,Kilometers,Type,Gearbox,Fuel,Status,CC,Color,Seating Capacity
0,12830792,2004 Nissan Pulsar ST-L N16 MY04,"$7,995",Nissan,Pulsar,ST-L,N16 MY04,2004,"120,083 km",Sedan,Automatic,Unleaded Petrol,Used,1769,Red,5
1,12825831,2022 BMW X3 M Competition M Steptronic M xDrive,"$178,900",BMW,X3 M,Competition M Steptronic M xDrive,,2022,"3,532 km",Wagon,Automatic,-,Demo,,,
2,12825829,2022 BMW 3 Series 25i,"$109,900",BMW,3 Series,25i,,2022,"3,544 km",OtherCar,Automatic,-,Demo,,White,
3,12825828,2022 BMW 3 Series 25i,"$110,900",BMW,3 Series,25i,,2022,"3,538 km",OtherCar,Automatic,-,Demo,,Blue,
4,12700658,2004 Mercedes-Benz Sprinter 316CDI SWB,"$54,890",Mercedes-Benz,Sprinter,316CDI SWB,,2004,"39,915 km",Van,Manual,Diesel,Used,2686,Green,3
5,12825833,2022 BMW 5 Series M550i xDrive Steptronic,"$169,900",BMW,5 Series,M550i xDrive Steptronic,,2022,"3,512 km",OtherCar,Automatic,-,Demo,,Black,
6,12732064,2019 BMW 3 Series 25i,"$92,900",BMW,3 Series,25i,,2019,"35,000 km",OtherCar,Automatic,-,Used,,Black,
7,12722550,2012 Mercedes-Benz Sprinter 313CDI LWB Hi Roof...,"$32,990",Mercedes-Benz,Sprinter,313CDI LWB Hi Roof,906 MY11,2012,"278,290 km",Van,Automatic,Diesel,Used,2143,White,2
8,12786212,2022 BMW 2 SERIES 230i Steptronic M Sport,"$79,900",BMW,2 SERIES,230i Steptronic M Sport,,2022,"3,511 km",Coupe,Automatic,-,Demo,,Black,
9,12786211,2022 BMW 2 SERIES 230i Steptronic M Sport,"$78,900",BMW,2 SERIES,230i Steptronic M Sport,,2022,"3,510 km",Coupe,Automatic,-,Demo,,Blue,


- Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

==> Mỗi dòng đại diện cho một chiếc xe được bán. Không có vấn đề mỗi dòng có ý nghĩa khác nhau. Các dòng đều mang một ý nghĩa như nhau là thể hiện chiếc xe đang được bán.
- Mỗi cột có ý nghĩa gì?

==>Mỗi cột là một thông số của xe. Ở đây tụi em sử dụng 16 cột dữ liệu, ý nghĩa của mỗi cột như sau:
- ID: Mã số của xe.
- Name: Tên xe.
- Price: Giá xe.
- Brand: Thương hiệu xe.
- Model: Đời xe.
- Variant: Biến thể xe. 
- Series: Series xe.
- Year: Năm sản xuất.
- Kilometers: Số km xe chạy được (xe mới thuộc tính này là 0).
- Type: Loại xe (Sedan, Wagon, Coupe,...).
- Gearbox: Hộp số (Tự động, thủ công,...).
- Fuel: Nhiên liệu (Dầu, xăng,...).
- Status: Trạng thái (Đã sử dụng, thử nghiệm,...).
- CC: Phân khối.
- Color: Màu xe.
- Seating Capacity: Số chỗ ngồi.

In [127]:
for col in df.columns:
    print(col,": ",type(col))

ID :  <class 'str'>
Name :  <class 'str'>
Price :  <class 'str'>
Brand :  <class 'str'>
Model :  <class 'str'>
Variant :  <class 'str'>
Series :  <class 'str'>
Year :  <class 'str'>
Kilometers :  <class 'str'>
Type :  <class 'str'>
Gearbox :  <class 'str'>
Fuel :  <class 'str'>
Status :  <class 'str'>
CC :  <class 'str'>
Color :  <class 'str'>
Seating Capacity :  <class 'str'>


- Mỗi cột hiện đang có kiểu dữ liệu gì? Khoảng biểu diễn của kiểu dữ liệu đó ra sao? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

==> Kiểu dữ liệu của cột: 
- ID: string.
- Name: string.
- Price: string.
- Brand: string.
- Model: string.
- Variant: string.
- Series: string.
- Year: string.
- Kilometers: string.
- Type: string.
- Gearbox: string.
- Fuel: string.
- Status: string.
- CC: string.
- Color: string.
- Seating Capacity: string.

Khoảng biểu diễn của kiểu dữ liệu đó ra sao? 

==>Do đều là string nên có phạm vi biểu diễn cụ thể.

Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

==> Có một số cột ta có thể chuyển từ string về dạng số, có thể là float để dễ dàng xử lý. Các cột đó là cột "Price", "Year", "Kilometers", "CC", "Seating Capacity".

Ngoài ra sẽ có những xe bị khuyết một số thuộc tính. Ở đây nhóm chúng em xử lý bằng cách xóa luôn những dòng bị khuyết dữ liệu.

Cell dưới dùng để xóa kí tự '$' ở cột "Price" và " km" ở cột Kilometers

In [128]:
cols=["Price", "Year", "Kilometers", "CC", "Seating Capacity"]

for i in range(len(df['Price'])):
    df['Price'][i]=df['Price'][i].replace('$','')
    df['Price'][i]=df['Price'][i].replace(',','')
for i in range(len(df['Kilometers'])):
    df['Kilometers'][i]=df['Kilometers'][i].replace(' km','')
    df['Kilometers'][i]=df['Kilometers'][i].replace(',','')


C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_2224/2925046440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i]=df['Price'][i].replace('$','')
C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_2224/2925046440.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i]=df['Price'][i].replace(',','')
C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_2224/2925046440.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Xóa các dòng bị khuyết.

In [129]:
for col in df.columns:
    df.drop(df.index[df[col] == '-'], inplace = True)
    df.drop(df.index[df[col] == ''], inplace = True)




Chuyển 5 cột trên từ string về float

In [131]:
df = df.astype({'Price':'float','Year':'float','Kilometers':'float','CC':'float','Seating Capacity':'float'})
